In [57]:
from DiscretizationSchemes import discretizationScheme, StockpriceGBM # consider different discretization schemes?
from intrinsicOptionValues import call, put # consider more payoffs?


# Page 18 (Lokeshwar): Bermudan put on a single asset

In [58]:
# set parameters
S0, K, r, sigma, T, M = 40, 40, 0.06, 0.2, 1, 10
p = 32
Np = 3*p + 1
mu = r # changing this should not make a difference??

In [59]:
# define phi for 1-dimensional vector
def phi(x: float):
  return max(x, 0)

In [60]:
# define output network
def G(beta_tm, S_tm):
  network_output = beta_tm[3*p]
  for i in range(p):
      network_output += beta_tm[2*p + i]*phi(beta_tm[i]*S_tm + beta_tm[p+i]) 

In [61]:
from helpers import forward, bs_put, bs_call
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [62]:
# compute continuation value Qtm (for ease of notation we write tm where the book writes t_{m-1})
def Qt_mminus1(beta_tm, S_tm, Deltat_mplus1):
  """
  Deltat_mplus1 = t_{m+1} - t_m
  """
  sumCondExp = beta_tm[3*p]
  for i in range(p):
    w_i, b_i, omega_i = beta_tm[i], beta_tm[p+i], beta_tm[2*p+i]
    if w_i >= 0 and b_i >= 0:
      condExp = forward(w_i*S_tm + b_i, r, Deltat_mplus1)
    elif w_i >= 0 and b_i < 0:
      condExp = w_i * bs_call(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i >= 0:
      condExp = -w_i * bs_put(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i < 0:
      condExp = 0
    sumCondExp += omega_i*condExp
  return sumCondExp


In [63]:
monitoring_dates = np.linspace(0,T,M+1)[1:]
N = 10000

In [64]:
def fitBeta(S_tm, V_tm, load_weights):

  # initialize model
  model = keras.Sequential()

  # input layer
  kernel_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  bias_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  model.add(keras.layers.Dense(p, input_dim=1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='relu'))

  # output layer
  kernel_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  bias_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  model.add(keras.layers.Dense(1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='linear'))
  
  # optimize using Adam
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt,loss='mean_squared_error')

  # reuse weights from previous timestep
  if load_weights==True:
    model.load_weights('model_weights.h5', by_name=False)

  # fit the model
  x = S_tm
  y = V_tm
  model.fit(x, y, epochs=1000, batch_size=int(N/10))

  # save weights for next timestep
  model.save_weights('model_weights.h5')
  return model

In [70]:
def genPaths(monitoring_dates, S0, mu, sigma, N):
  S = []
  for n in range(N):
    path = discretizationScheme(monitoring_dates, S0, mu, sigma)
    S.append(path)
  S = np.matrix(S) # (n,m)
  return S

In [65]:
# generate paths
S = genPaths(monitoring_dates, S0, mu, sigma, N)

# evaluate final time option value
V = np.zeros((N,M))
for n in range(N):
  S_T = S[n,M-1]
  V[n,M-1] = max(K-S_T, 0)

# keep track of Q to see what goes wrong
holdValues = np.zeros((N,M-1))

# no weights to load before the first iteration
load_weights = False

for m in range(M,0,-1):

  # fit the network
  fit_tm = fitBeta(S[:,m-1],V[:,m-1],load_weights=load_weights)

  # load weights next iteration
  load_weights = True

  # preprocess fitted weights for Q
  weights = fit_tm.get_weights()
  w1 = np.array(weights[0][0])
  b1 = np.array(weights[1])
  w2 = np.array(weights[2]).flatten()
  b2 = np.array(weights[3])
  beta_tm = np.concatenate((w1,b1,w2,b2))

  # stop when t=t0
  if m==0:
    break
  for n in range(N):
    Deltat_m = monitoring_dates[m-1] - monitoring_dates[m-2]
    St_mminus1 = S[n,m-2]
    holdValue = Qt_mminus1(beta_tm, St_mminus1, Deltat_m)
    holdValues[n, m-2] = holdValue
    V[n,m-2] = max(St_mminus1 - K, holdValue)


Epoch 1/1000
10/10 [==============================] - 0s 517us/step - loss: 127845.0938
Epoch 2/1000
10/10 [==============================] - 0s 455us/step - loss: 117908.7578
Epoch 3/1000
10/10 [==============================] - 0s 488us/step - loss: 108678.6797
Epoch 4/1000
10/10 [==============================] - 0s 432us/step - loss: 100235.2656
Epoch 5/1000
10/10 [==============================] - 0s 466us/step - loss: 92691.4297
Epoch 6/1000
10/10 [==============================] - 0s 447us/step - loss: 85770.6172
Epoch 7/1000
10/10 [==============================] - 0s 483us/step - loss: 79421.7578
Epoch 8/1000
10/10 [==============================] - 0s 543us/step - loss: 73655.5625
Epoch 9/1000
10/10 [==============================] - 0s 547us/step - loss: 68326.6719
Epoch 10/1000
10/10 [==============================] - 0s 474us/step - loss: 63396.5391
Epoch 11/1000
10/10 [==============================] - 0s 415us/step - loss: 58825.4844
Epoch 12/1000
10/10 [================

Epoch 1/1000
10/10 [==============================] - 0s 498us/step - loss: 6.1506
Epoch 2/1000
10/10 [==============================] - 0s 467us/step - loss: 5.4978
Epoch 3/1000
10/10 [==============================] - 0s 467us/step - loss: 5.3889
Epoch 4/1000
10/10 [==============================] - 0s 467us/step - loss: 5.3507
Epoch 5/1000
10/10 [==============================] - 0s 466us/step - loss: 5.3521
Epoch 6/1000
10/10 [==============================] - 0s 443us/step - loss: 5.3336
Epoch 7/1000
10/10 [==============================] - 0s 461us/step - loss: 5.3352
Epoch 8/1000
10/10 [==============================] - 0s 448us/step - loss: 5.3123
Epoch 9/1000
10/10 [==============================] - 0s 463us/step - loss: 5.3086
Epoch 10/1000
10/10 [==============================] - 0s 435us/step - loss: 5.2975
Epoch 11/1000
10/10 [==============================] - 0s 430us/step - loss: 5.2784
Epoch 12/1000
10/10 [==============================] - 0s 499us/step - loss: 5.2737
E

Epoch 1/1000
10/10 [==============================] - 0s 486us/step - loss: 0.3278
Epoch 2/1000
10/10 [==============================] - 0s 465us/step - loss: 0.1041
Epoch 3/1000
10/10 [==============================] - 0s 438us/step - loss: 0.0438
Epoch 4/1000
10/10 [==============================] - 0s 468us/step - loss: 0.0131
Epoch 5/1000
10/10 [==============================] - 0s 473us/step - loss: 0.0046
Epoch 6/1000
10/10 [==============================] - 0s 473us/step - loss: 0.0018
Epoch 7/1000
10/10 [==============================] - 0s 469us/step - loss: 6.6843e-04
Epoch 8/1000
10/10 [==============================] - 0s 514us/step - loss: 2.3014e-04
Epoch 9/1000
10/10 [==============================] - 0s 428us/step - loss: 7.9572e-05
Epoch 10/1000
10/10 [==============================] - 0s 433us/step - loss: 3.4167e-05
Epoch 11/1000
10/10 [==============================] - 0s 479us/step - loss: 1.2837e-05
Epoch 12/1000
10/10 [==============================] - 0s 453us/s

Epoch 1/1000
10/10 [==============================] - 0s 560us/step - loss: 0.3243
Epoch 2/1000
10/10 [==============================] - 0s 458us/step - loss: 0.1028
Epoch 3/1000
10/10 [==============================] - 0s 450us/step - loss: 0.0432
Epoch 4/1000
10/10 [==============================] - 0s 504us/step - loss: 0.0130
Epoch 5/1000
10/10 [==============================] - 0s 436us/step - loss: 0.0045
Epoch 6/1000
10/10 [==============================] - 0s 400us/step - loss: 0.0018
Epoch 7/1000
10/10 [==============================] - 0s 459us/step - loss: 6.5590e-04
Epoch 8/1000
10/10 [==============================] - 0s 426us/step - loss: 2.2733e-04
Epoch 9/1000
10/10 [==============================] - 0s 436us/step - loss: 7.9802e-05
Epoch 10/1000
10/10 [==============================] - 0s 446us/step - loss: 3.4835e-05
Epoch 11/1000
10/10 [==============================] - 0s 435us/step - loss: 1.2870e-05
Epoch 12/1000
10/10 [==============================] - 0s 428us/s

Epoch 1/1000
10/10 [==============================] - 0s 490us/step - loss: 0.3215
Epoch 2/1000
10/10 [==============================] - 0s 466us/step - loss: 0.1019
Epoch 3/1000
10/10 [==============================] - 0s 460us/step - loss: 0.0424
Epoch 4/1000
10/10 [==============================] - 0s 453us/step - loss: 0.0126
Epoch 5/1000
10/10 [==============================] - 0s 451us/step - loss: 0.0044
Epoch 6/1000
10/10 [==============================] - 0s 486us/step - loss: 0.0017
Epoch 7/1000
10/10 [==============================] - 0s 459us/step - loss: 6.4753e-04
Epoch 8/1000
10/10 [==============================] - 0s 458us/step - loss: 2.2337e-04
Epoch 9/1000
10/10 [==============================] - 0s 431us/step - loss: 7.7440e-05
Epoch 10/1000
10/10 [==============================] - 0s 414us/step - loss: 3.3978e-05
Epoch 11/1000
10/10 [==============================] - 0s 446us/step - loss: 1.2761e-05
Epoch 12/1000
10/10 [==============================] - 0s 430us/s

Epoch 1/1000
10/10 [==============================] - 0s 491us/step - loss: 0.3076
Epoch 2/1000
10/10 [==============================] - 0s 469us/step - loss: 0.0951
Epoch 3/1000
10/10 [==============================] - 0s 425us/step - loss: 0.0404
Epoch 4/1000
10/10 [==============================] - 0s 455us/step - loss: 0.0127
Epoch 5/1000
10/10 [==============================] - 0s 455us/step - loss: 0.0043
Epoch 6/1000
10/10 [==============================] - 0s 438us/step - loss: 0.0016
Epoch 7/1000
10/10 [==============================] - 0s 440us/step - loss: 5.8298e-04
Epoch 8/1000
10/10 [==============================] - 0s 471us/step - loss: 2.0297e-04
Epoch 9/1000
10/10 [==============================] - 0s 530us/step - loss: 8.0704e-05
Epoch 10/1000
10/10 [==============================] - 0s 426us/step - loss: 3.4529e-05
Epoch 11/1000
10/10 [==============================] - 0s 440us/step - loss: 9.9156e-06
Epoch 12/1000
10/10 [==============================] - 0s 468us/s

Epoch 1/1000
10/10 [==============================] - 0s 496us/step - loss: 0.3077
Epoch 2/1000
10/10 [==============================] - 0s 444us/step - loss: 0.0958
Epoch 3/1000
10/10 [==============================] - 0s 425us/step - loss: 0.0406
Epoch 4/1000
10/10 [==============================] - 0s 417us/step - loss: 0.0125
Epoch 5/1000
10/10 [==============================] - 0s 478us/step - loss: 0.0043
Epoch 6/1000
10/10 [==============================] - 0s 437us/step - loss: 0.0016
Epoch 7/1000
10/10 [==============================] - 0s 449us/step - loss: 5.9445e-04
Epoch 8/1000
10/10 [==============================] - 0s 487us/step - loss: 2.0444e-04
Epoch 9/1000
10/10 [==============================] - 0s 431us/step - loss: 7.7244e-05
Epoch 10/1000
10/10 [==============================] - 0s 399us/step - loss: 3.4021e-05
Epoch 11/1000
10/10 [==============================] - 0s 464us/step - loss: 1.0401e-05
Epoch 12/1000
10/10 [==============================] - 0s 434us/s

Epoch 1/1000
10/10 [==============================] - 0s 484us/step - loss: 0.2461
Epoch 2/1000
10/10 [==============================] - 0s 460us/step - loss: 0.0678
Epoch 3/1000
10/10 [==============================] - 0s 457us/step - loss: 0.0280
Epoch 4/1000
10/10 [==============================] - 0s 493us/step - loss: 0.0108
Epoch 5/1000
10/10 [==============================] - 0s 488us/step - loss: 0.0039
Epoch 6/1000
10/10 [==============================] - 0s 478us/step - loss: 0.0015
Epoch 7/1000
10/10 [==============================] - 0s 447us/step - loss: 5.5271e-04
Epoch 8/1000
10/10 [==============================] - 0s 501us/step - loss: 1.6964e-04
Epoch 9/1000
10/10 [==============================] - 0s 452us/step - loss: 5.9577e-05
Epoch 10/1000
10/10 [==============================] - 0s 412us/step - loss: 2.6227e-05
Epoch 11/1000
10/10 [==============================] - 0s 435us/step - loss: 7.3666e-06
Epoch 12/1000
10/10 [==============================] - 0s 438us/s

Epoch 1/1000
10/10 [==============================] - 0s 487us/step - loss: 0.3088
Epoch 2/1000
10/10 [==============================] - 0s 450us/step - loss: 0.0985
Epoch 3/1000
10/10 [==============================] - 0s 421us/step - loss: 0.0411
Epoch 4/1000
10/10 [==============================] - 0s 516us/step - loss: 0.0123
Epoch 5/1000
10/10 [==============================] - 0s 427us/step - loss: 0.0043
Epoch 6/1000
10/10 [==============================] - 0s 428us/step - loss: 0.0017
Epoch 7/1000
10/10 [==============================] - 0s 472us/step - loss: 6.3426e-04
Epoch 8/1000
10/10 [==============================] - 0s 438us/step - loss: 2.1912e-04
Epoch 9/1000
10/10 [==============================] - 0s 454us/step - loss: 7.5250e-05
Epoch 10/1000
10/10 [==============================] - 0s 481us/step - loss: 3.2333e-05
Epoch 11/1000
10/10 [==============================] - 0s 431us/step - loss: 1.2470e-05
Epoch 12/1000
10/10 [==============================] - 0s 435us/s

Epoch 1/1000
10/10 [==============================] - 0s 483us/step - loss: 0.3046
Epoch 2/1000
10/10 [==============================] - 0s 480us/step - loss: 0.0966
Epoch 3/1000
10/10 [==============================] - 0s 445us/step - loss: 0.0404
Epoch 4/1000
10/10 [==============================] - 0s 463us/step - loss: 0.0121
Epoch 5/1000
10/10 [==============================] - 0s 475us/step - loss: 0.0042
Epoch 6/1000
10/10 [==============================] - 0s 435us/step - loss: 0.0016
Epoch 7/1000
10/10 [==============================] - 0s 445us/step - loss: 6.1132e-04
Epoch 8/1000
10/10 [==============================] - 0s 478us/step - loss: 2.1037e-04
Epoch 9/1000
10/10 [==============================] - 0s 444us/step - loss: 7.3463e-05
Epoch 10/1000
10/10 [==============================] - 0s 412us/step - loss: 3.2063e-05
Epoch 11/1000
10/10 [==============================] - 0s 441us/step - loss: 1.1759e-05
Epoch 12/1000
10/10 [==============================] - 0s 450us/s

In [66]:
print(V[4,:])

[ 0.         -0.68446012 -0.64350011 -1.19734301 -1.26412574 -1.51620616
 -0.66629803  0.06071546 -1.83464223  0.        ]


In [67]:
print(S[4,:])

[[40.         40.68446012 40.64350011 41.19734301 41.26412574 41.51620616
  40.66629803 39.93928454 41.83464223 42.18321741]]


In [68]:
print(holdValues[4,:])

[-44.31999289 -48.12191997 -47.97233602 -51.01726924 -51.36393994
 -52.81591354 -47.96087768 -43.90256344 -67.70142948]
